# Melusine + Arta's Workshop

## Business Context &#128221;

Your are working for an insurance company.

Customers are contacting your company for two main reasons:

* **Subscriptions:** subscribe, modify or close insurance policies.
* **Claims:** initiate or follow the claim process of a covered loss (car, housing, etc).

Many employees work on processing client emails through a custom user interface (i.e., a business application).

## Data Science Context &#x1F575;

You are a Data Scientist in charge of optimizing the email pre-processing (i.e., before being processed by a human).

For that purpose your goal is to automatize:

* The definition of the *priority* of an email to: `low`, `normal` or `high`.
* The categorization of an email into two target business units: `claim` and `contracts`.
* The reply in case of a thank you email.

Two open-source packages must be used:

* [Melusine](https://github.com/MAIF/melusine): Framework for automatic email processing.

> Use: create a detection pipeline and use it to identify relevant patterns in the emails.

* [Arta](https://github.com/MAIF/arta): A python rules engine

> Use: apply rules to translate the detection outputs into clear actions or extra data needed by the business application.


First, let's make sure those packages are installed in your *python virtual environment*.

In [ ]:
# !pip install 'melusine>=3.2' arta

## Dataset

During this workshop, you will use a demo dataset containing a variety of emails in French and in English.
In the first part, you can explore the dataset, then you will use **Melusine** and **Arta** to qualify (i.e., pre-process) the emails.


Let's import the dataset and take a sneak peak.

In [ ]:
import pandas as pd

df = pd.read_json("workshop_email_data.json")
df.head()

## Melusine: a processing pipeline

### Email Preprocessing

**Melusine** is a python framework to automatize emails processing.

One of the key feature is the default *preprocessing pipeline* which does the following:

1. **Cleaning**: remove undesired characters, replace non-breaking spaces, handle specific combinations (ex: "œ" => "oe").
1. **Email Segmentation**: separate an email conversation with mutiple replies and transfers into a list of individual messages.
1. **Message Tagging**: tag the different parts of the message (typically to get rid of unwanted footers and signature text).

These steps can be edited through configuration files or class inheritance but for the sake of this workshop we will stick with the default preprocessing pipeline.

Let's import and run this pipeline on our data.

In [ ]:
from melusine.pipeline import MelusinePipeline


pipe = MelusinePipeline.from_config("preprocessing_pipeline")

In [ ]:
preprocessed_df = pipe.transform(df)

# Look at last columns
preprocessed_df.head()

####  &#x1F449; Your turn &#x1F477;

The first email in the DataFrame is composed of two messages (initial message and forwarded message). The raw email string looks like this:

In [ ]:
print(preprocessed_df.iloc[0]["body"])

The preprocessing pipeline created a new column `messages` which is a list of `Message` objects.
Hint 1 : You can use print on a `Message` object for a user-friendly display.
Hint 2 : The `extract_parts` and `extract_text` methods can be useful to extract the text of a message.
Check out the [Message](https://github.com/MAIF/melusine/blob/master/melusine/message.py) class for more details.

Inspect the message corpus:

* Print the first and second messages of the first email in the dataset.
* Extract the part tagged as "BODY" of any message of your choice.


### The MelusineRegex

The `MelusineRegex` is a convenient class to simplify detections using **regexes**. Main features are:

* Name "regex patterns" for readable and explainable results.
* Easily define patterns to be ignored and patterns that prevent matching (forget complex and unreadable negative look behind mecanisms).
* Keep matching and non-matching exemples close to the regex definition to avoid loosing track of what the regex does.

We will provide a working regex for this workshop: `EmergencyRegex`. This one will be used later to detect urgent requests in the emails.

*N.B: You can check the code of the class [MelusineRegex](https://github.com/MAIF/melusine/blob/29815b578ba852b7c79116a926ef0a0e3bd0e1d5/melusine/base.py#L314), in particular docstrings and methods' type hints.*

####  &#x1F449; Your turn &#x1F477;

When intantiating the `EmergencyRegex` class, it will be tested against every element of the `match_list` (match expected) and the `no_match_list` (no match expected).
An error has been introduced in the regexes, try to find it and correct it.
At instantitation, if the test passes, nothing will be displayed, otherwise an error will be raised.

In [ ]:
from melusine.base import MelusineRegex


class EmergencyRegex(MelusineRegex):
    
    @property
    def positive(self):
        """At least one of these patterns has to match for a global match of the MelusineRegex"""
        return {
            "regex_1": "urggen(t|ce)|emergency",
            "regex_2": r"as soon as possible", 
            "regex_3": r"(d[èe]s que|le plus vite|aussi vite que) possible",
        }
    
    @property
    def negative(self):
        """Any matching pattern will prevent the global match of the MelusineRegex"""
        return r"urgences"
    
    @property
    def match_list(self):
        """Text exemples that MUST be matched by the MelusineRegex"""
        return [
            "C'est urgent",
            "Ceci est une URGENCE",
            "Call me as soon as possible",
        ]
    
    @property
    def no_match_list(self):
        """Text exemples that MUST NOT be matched by the MelusineRegex"""
        return [
            "Je travaille dans le service des urgences",
        ]

In [ ]:
# Instantiate your new MelusineRegex
reg = EmergencyRegex()

### The MelusineDetector

Business requests may become more and more complex over time.

E.g., "Could you handle this weird edge case when the email is sent from an autonomous vehicle?"

The objective of the `MelusineDetector` is to standardise how *detections* are performed on emails and restrain the technical debt that occurs during the life of an application.

The key elements of a `MelusineDetector` are:

1. **Declaration of input and output columns**: missing columns in the input DataFrame can be identified early-on => Keep track of which detector created a column.
1. **pre_detect()**: processing to be done prior to detection. Typically assemble the text of interest.
1. **detect()**: performed the core detection using regex, machine learning model or heuristics.
1. **post_detect()**: combine the detection outputs to product the final detector result.

You can find more details in the Melusine documentation on [MelusineDetector tutorials](https://maif.github.io/melusine/tutorials/05a_MelusineDetectors).

Install unidecode, needed by the detector:

In [ ]:
# !pip install unidecode

In [ ]:
from melusine.base import MelusineDetector
from unidecode import unidecode


class EmergencyDetector(MelusineDetector):
    """A detector to detect emergency in emails"""

    def __init__(self, name="emergency_detector"):
        self.regex = EmergencyRegex()

    @property
    def input_columns(self):
        """Input columns required by the detector"""
        return ["messages", "header"]

    @property
    def output_columns(self):
        """Output columns created by the detector"""
        return ["emergency_detector_result"]

    def pre_detect(self, row, debug_mode=False):
        """Data transformations prior to the core detection"""
        last_body = row["messages"][0].extract_text(target_tags=["BODY", "THANKS"])
        row["effective_text"] = row["header"] + "\n" + unidecode(last_body).lower()
        return row

    def detect(self, row, debug_mode=False):
        """Core detection method"""
        match_data = self.regex(row["effective_text"])
        row["emergency_detector_result"] = match_data["match_result"]
        
        if debug_mode:
            row[self.DEBUG_DICT_COL] = match_data
            
        return row
    
    def post_detect(self, row, debug_mode=False):
        """Data transformations posterior to the core detection"""
        return row

In [ ]:
emergency_detector = EmergencyDetector()

#### Running the Detector on an Email Corpus

Run the detector on the email corpus and checkout the emergency detector's result (`emergency_detector_result` column).

In [ ]:
emergency_df = emergency_detector.transform(preprocessed_df)
emergency_df.head()

####  &#x1F449; Your turn &#x1F477;
Now, implement a `ThanksRegex` and integrate it to a `ThanksDetector`.
(Replace the `?` with the correct code in both the `ThanksRegex` and `ThanksDetector` classes.)

In [ ]:
class ThanksRegex(MelusineRegex):

    @property
    def positive(self):
        return {
            "thanks_fr": ?,
            "thanks_en": ?,
        }

    @property
    def negative(self):
        return {
            "negative_thanks_fr": ?,
        }

    @property
    def match_list(self):
        return [
            ?
        ]

    @property
    def no_match_list(self):
        return [
            ?
        ]

In [ ]:
class ThanksDetector(MelusineDetector):

    def __init__(self, name="thanks_detector"):
        self.regex = ?

    @property
    def input_columns(self):
        return [ ? ]

    @property
    def output_columns(self):
        return [ ? ]

    def pre_detect(self, row, debug_mode=False):
        ?
        return row

    def detect(self, row, debug_mode=False):
        ?

        return row

    def post_detect(self, row, debug_mode=False):
        return row

In [ ]:
thanks_detector = ThanksDetector()

Now, implement a `RoutingRegex` and integrate it to a `RoutingDetector`.

In [ ]:
class RoutingRegex(MelusineRegex):

    ?

In [ ]:
class RoutingDetector(MelusineDetector):

    ?


In [ ]:
routing_detector = RoutingDetector()

### The MelusinePipeline

The `MelusinePipeline` object (inheriting from the sklearn Pipeline object) lets you assemble all the detectors into a pipeline that can be executed with a simple `transform()` method. It also brings features like:
   * input columns check (raises an error if mandatory input columns are missing)
   * pipeline load from (YAML) configuration files.

Let's assemble the pipeline with the detectors we have created.

In [ ]:
from melusine.pipeline import MelusinePipeline

pipe = MelusinePipeline([
    ("emergency_detector", emergency_detector),
    ("thanks_detector", thanks_detector),
    ("routing_detector", routing_detector),
])

Finally, let's run the pipeline on our DataFrame:

In [ ]:
df_transformed = pipe.transform(preprocessed_df)
df_transformed.head(21)

## Arta: a rules engine

### Use Case

Thanks to the previous **Melusine** pipeline, we are able to process emails and get the following extra details:

* Is it an urgent email?
* Is it about claims or contracts?
* Is it a thank you email?

But this new data is not enough. Remember, we need to define for each email a *priority* and if we need an *automatic reply* or not.

After talking with the people in charge of handling the customers emails, we defined the following business rules:

#### Email Priority

1. If an email is urgent, regardless of the business unit (claims or contracts), the priority should be set to "high".
2. If an email is a thank you email for the claims, priority should be set to "low".
3. Other cases should be set to "medium".

#### Automatic Reply

1. If the email is about contracts and is a thank you email, you should send the automatic reply which id is "THANK_U_4_YOUR_THANK_U".
2. Do nothing for other cases.


**Arta**`s goal is to simplify *rules definition* and *rules execution* in order to produce the expected outputs (priority and automatic reply here).

To do that, we simply need to:

1. Prepare the input data.
1. Define the business rules.
1. Implement the needed functions.
1. Use the rules engine to apply the rules.

### Input Data

In **Arta**, `input_data` is the data fed to the rules engine. In other words, it is the data on which the rules are applied.

It must be a [mapping](https://docs.python.org/3/glossary.html#term-mapping) object (i.e., a dictionary like object).

This dictionary could be an example:

```python
input_data = {
    "id": 42,
    "cleaned_body": "Hi, I tried to reach you but unfortunately blah blah...", 
    "business": "CLAIM",
    "emergency": True, 
    "thank_you": False,
}
```

First, we are going to develop a function that maps the Melusine pipeline's output to the Arta's input data.

####  &#x1F449; Your turn &#x1F477;

Implement a function to convert a line (i.e., a `pandas.Series`) of the `DataFrame` referenced in the `processed_df` variable into a python dictionary with the same schema as below.

In [ ]:
def convert_to_dict(row):
    ?

In [ ]:
# Test
input_data = convert_to_dict(df_transformed.iloc[0])
print(input_data)

### Rule Group

In **Arta**, a **rule group** is grouping different **rules** with a common output (generally, but not always).

In our case, we could define 2 groups:

1. `priority`
1. `auto_reply`

**Rule groups** are made of **rules** and they are bundled into a **rule set**. 
All of these are defined in a configuration file (i.e., *YAML file*) as you can see in the [documentation](https://maif.github.io/arta/how_to/#rule-set-and-rule-group).

####  &#x1F449; Your turn &#x1F477;

Define above rules using the following template. You can name the *validation functions* as you want, you will implement them later.

*Hint:  use the [standard conditions](https://maif.github.io/arta/how_to/#simple-condition).*

```yaml
---
rules:
  default_rule_set:
    admission:
# Complete below
        

actions_source_modules:
  - assets.actions
```

When your are done, copy and paste these rules into a new YAML file: `./assets/rules.yaml` (file name of your choice).

### Action Functions

A *rule* is made of *conditions* and one *action*. When conditions are validated (i.e., True) the *action* is executed.

In fact, *actions* are just regular python *function* and they are named **action functions**.

If you go back to the YAML definition of your rules, you can see:

```yaml
        action: set_priority
        action_parameters:
          value: MEDIUM
```

* `action:` is the name of your action function.
* `action_parameters:` are the names and the values of the arguments passed to the function when executed.

####  &#x1F449; Your turn &#x1F477;

Look at your rules and list the validation functions that you need to implement:

* ...
* ...

Now, implement them in the following python module `rules/actions.py`. 

*Hint: look at the following [documentation page](https://maif.github.io/arta/a_simple_example/#actions).*

### Rule Engine

Last but not least, we need to apply the rules on our data, it is the purpose of the rules engine.

####  &#x1F449; Your turn &#x1F477;

Instanciate a rules engine and apply the rules on the **Melusine** pipeline results for one email.

*Hint: [Usage documentation](https://maif.github.io/arta/how_to/#usage).*

####  &#x1F449; Your turn &#x1F477;

Apply on all emails and print their results.

Compare the obtained results with expected ones.

## End

Now we can feed the employee business application with the emails and the needed extra informations. Automatic replies and prioritization will speed up the email processing.